<h2>Our models evaluation</h2>

Our models used robrta-base and robrta-base-boolq that are further trained on our auto-generated questions.
our evalution citeria is accuracy and f-score of the model and confusion matrix to visualize claasfication patterns.
Both auto-generated and manually generated questions are tested.

In [1]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, Adafactor
from transformers import T5ForConditionalGeneration,T5Tokenizer
import seaborn as sns
import matplotlib.pyplot as plt
import random
import torchvision.models as models
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

KeyboardInterrupt: 

In [ ]:
def predict(question, passage, model):
    sequence = tokenizer.encode_plus(question, passage, return_tensors="pt")['input_ids'].to(device)
  
    logits = model(sequence)[0]
    probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
    proba_yes = round(probabilities[1], 2)
    proba_no = round(probabilities[0], 2)
    if(proba_yes > proba_no):
        return 1
    else:
        return 0


# test on manually Generated Questions

In [ ]:
df = pd.read_csv('../data/manuallyGeneratedQuestions.csv', header=0)
df.head()

In [ ]:
# dataset example
Recipes = list(df["Recipe"]) 
print(Recipes[0])
print()
Questions = list(df["Question"])
labels = list(df["label"])
print(Questions[0], ", answer: ",labels[0])


print("total rows: ", len(Recipes))

# roberta-base model

In [ ]:
# type(y_pred[0])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
answering_model = AutoModelForSequenceClassification.from_pretrained("roberta-base")
answering_model.load_state_dict(torch.load('../models/2ksamplesAcc55.h5', map_location=device))
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
answering_model.to(device)
y_true = []
y_pred = []

tested = 0

# Predict the answer from the passage and the question
for i in range(len(Recipes)):
    #answer the Q
    tested +=1
    ans = predict(question=Questions[i], passage=Recipes[i], model=answering_model)
    y_pred.append(ans)
    

print("Done... ", tested)
print("len true: ", len(labels))
print("len pred: ", len(y_pred))

acc = 0
for i in range(len(labels)):
    if(labels[i] == y_pred[i]):
        acc +=1
print("acc: ", acc/len(labels))
print()

# for index in range(len(labels)):
#     labels[index] = str(labels[index])
cf_matrix = confusion_matrix(labels, y_pred)
print((cf_matrix/np.sum(cf_matrix)*100))


ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')
ax.set_title('Confusion Matrix \n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

print(f1_score(labels, y_pred, average='macro',zero_division=1))
print(f1_score(labels, y_pred, average='micro',zero_division=1))
print(f1_score(labels, y_pred, average='weighted',zero_division=1))

# roberta-base-boolq model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
answering_model = AutoModelForSequenceClassification.from_pretrained("roberta-base")
answering_model.load_state_dict(torch.load('models/robertaBool6175.h5', map_location=device))
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

y_true = []
y_pred = []

tested = 0

# Predict the answer from the passage and the question
for i in range(len(Recipes)):
    #answer the Q
    tested +=1
    ans = predict(question=Questions[i], passage=Recipes[i], model=answering_model)
    y_pred.append(ans)
    

 

print("Done... ", tested)
print("len true: ", len(labels))
print("len pred: ", len(y_pred))

acc = 0
for i in range(len(labels)):
    if(labels[i] == y_pred[i]):
        acc +=1
print("acc: ", acc/len(labels))
print()

cf_matrix = confusion_matrix(labels, y_pred)
print((cf_matrix/np.sum(cf_matrix)*100))


ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')
ax.set_title('Confusion Matrix \n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

print(f1_score(labels, y_pred, average='macro',zero_division=1))
print(f1_score(labels, y_pred, average='micro',zero_division=1))
print(f1_score(labels, y_pred, average='weighted',zero_division=1))